##**Librerias y dependencias necesarias**


In [39]:
from google.colab import drive #Acceso Drive
import os #Acceder a directorios
import pandas as pd #manejo de dataframes
from IPython.display import Audio #reproducir audio

##**Acceso al drive y ubicacion archivos**

In [40]:
#Damos acceso a colab a nuestro Drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


**Debemos asegurarnos de que la carpeta de las grabaciones compartida tenga un acceso directo creado en nuestro propio Drive.**

In [41]:
#Nos ubicamos en la direccion de interes
#Alli estaran las imagente fuente y las destino
dir='/content/drive/MyDrive/Jaguas_2018' #direccion raiz de nuestro proyecto actual
os.chdir(dir) #Ubicarnos en el directorio
print("Ubicados en :",dir)
!ls

Ubicados en : /content/drive/MyDrive/Jaguas_2018
G03_m  G07_m  G13_m  G19_m  G25_m  G29_m  G36_m  G41_m	G49_m  G52_m  G58_m
G04_m  G08_m  G15_m  G23_m  G27_m  G34_m  G37_m  G46_m	G50_m  G54_m
G06_m  G09_m  G17_m  G24_m  G28_m  G35_m  G40_m  G47_m	G51_m  G57_m


In [42]:
"""Cuenta la cantidad de archivos en cada carpeta de un directorio"""
total_archivos = 0
for raiz, carpetas, archivos in os.walk(dir):
  carpetas.sort()
  # Obtener la cantidad de archivos en la carpeta actual
  cantidad_archivos = len(archivos)  
  # Sumar la cantidad de archivos actual a la variable total_archivos
  total_archivos += cantidad_archivos      
  # Imprimir el resultado
  print(f"Grabadora: {os.path.basename(raiz)} Cantidad Grabaciones: {cantidad_archivos}")
print(f"Total grabaciones de todas las grabadoras: {total_archivos}")

Grabadora: Jaguas_2018 Cantidad Grabaciones: 0
Grabadora: G03_m Cantidad Grabaciones: 565
Grabadora: G04_m Cantidad Grabaciones: 418
Grabadora: G06_m Cantidad Grabaciones: 887
Grabadora: G07_m Cantidad Grabaciones: 621
Grabadora: G08_m Cantidad Grabaciones: 648
Grabadora: G09_m Cantidad Grabaciones: 908
Grabadora: G13_m Cantidad Grabaciones: 636
Grabadora: G15_m Cantidad Grabaciones: 874
Grabadora: G17_m Cantidad Grabaciones: 870
Grabadora: G19_m Cantidad Grabaciones: 886
Grabadora: G23_m Cantidad Grabaciones: 555
Grabadora: G24_m Cantidad Grabaciones: 555
Grabadora: G25_m Cantidad Grabaciones: 645
Grabadora: G27_m Cantidad Grabaciones: 644
Grabadora: G28_m Cantidad Grabaciones: 553
Grabadora: G29_m Cantidad Grabaciones: 553
Grabadora: G34_m Cantidad Grabaciones: 664
Grabadora: G35_m Cantidad Grabaciones: 627
Grabadora: G36_m Cantidad Grabaciones: 545
Grabadora: G37_m Cantidad Grabaciones: 541
Grabadora: G40_m Cantidad Grabaciones: 555
Grabadora: G41_m Cantidad Grabaciones: 553
Grabado

##**Creando el Data Frame**

In [43]:
# Crear un dataframe vacío para guardar la información de los archivos
df_archivos = pd.DataFrame(columns=['Recorder', 'Date', 'Time', 'Path'])

# Ruta del directorio principal que contiene las carpetas y archivos
ruta_principal = '/content/drive/MyDrive/Jaguas_2018'

# Recorrer las carpetas y archivos dentro de la ruta principal
for raiz, carpetas, archivos in os.walk(ruta_principal):
    carpetas.sort()
    # Recorrer los archivos de cada carpeta
    for archivo in archivos:
        # Obtener la ruta completa del archivo
        ruta_archivo = os.path.join(raiz, archivo)
        
        # Obtener el nombre del archivo sin la extensión
        nombre_archivo, extension = os.path.splitext(archivo)
        
        # Separar el nombre del archivo en partes utilizando el caracter "_"
        partes_nombre = nombre_archivo.split("_")
        
        # Obtener los valores deseados de cada parte del nombre del archivo
        recorder = partes_nombre[0]
        fecha = partes_nombre[1]
        hora = partes_nombre[2]
        
        # Crear un diccionario con los valores del archivo actual
        datos_archivo = {
            'Recorder': recorder,
            'Date': fecha,
            'Time': hora,
            'Path': ruta_archivo
        }
        
        # Agregar el diccionario al dataframe
        df_archivos = df_archivos.append(datos_archivo, ignore_index=True)

# Convertir la columna 'Date' al formato 'yyyy/mm/dd'
df_archivos['Date'] = df_archivos['Date'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d').strftime('%Y/%m/%d'))

# Convertir la columna 'Time' al formato 'hh:mm:ss'
df_archivos['Time'] = df_archivos['Time'].apply(lambda x: pd.to_datetime(x, format='%H%M%S').strftime('%H:%M:%S'))

#Cargamos informacion adicional de las grabadoras
df = pd.read_csv('/content/drive/MyDrive/Bio_Acustica/Datos.csv')

#hacemos un Marge con el dataframe anteriormente creado, a base de recorrer los archivos
df_archivos = pd.merge(df_archivos, df, on="Recorder")

In [47]:
df_archivos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20068 entries, 0 to 20067
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Recorder   20068 non-null  object 
 1   Date       20068 non-null  object 
 2   Time       20068 non-null  object 
 3   Path       20068 non-null  object 
 4   Municipio  20068 non-null  object 
 5   Vereda     20068 non-null  object 
 6   Latitud    20068 non-null  float64
 7   Longitud   20068 non-null  float64
 8   Elevacion  20068 non-null  int64  
 9   Habitat    20068 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.7+ MB


In [44]:
df_archivos

,Recorder,Date,Time,Path,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat
0,G03,2018/05/11,12:00:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
1,G03,2018/05/11,12:16:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
2,G03,2018/05/11,12:32:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
3,G03,2018/05/11,12:48:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
4,G03,2018/05/11,13:04:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
...,...,...,...,...,...,...,...,...,...,...
20063,G58,2018/07/04,07:00:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.35,-749.696,1072,Forest
20064,G58,2018/07/04,06:45:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.35,-749.696,1072,Forest
20065,G58,2018/07/04,07:15:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.35,-749.696,1072,Forest
20066,G58,2018/07/04,07:30:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.35,-749.696,1072,Forest


In [45]:
df_archivos["Recorder"].value_counts().sort_index()

G03    565
G04    418
G06    887
G07    621
G08    648
G09    908
G13    636
G15    874
G17    870
G19    886
G23    555
G24    555
G25    645
G27    644
G28    553
G29    553
G34    664
G35    627
G36    545
G37    541
G40    555
G41    553
G46    735
G47    804
G49    801
G50    620
G51    408
G52    465
G54    576
G57    621
G58    735
Name: Recorder, dtype: int64

In [46]:
df_archivos.to_csv('/content/drive/MyDrive/Bio_Acustica/df_archivos.csv', index=False)

#**Cargar Data Frame**


In [48]:
df_archivos = pd.read_csv('/content/drive/MyDrive/Bio_Acustica/df_archivos.csv')

In [49]:
df_archivos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20068 entries, 0 to 20067
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Recorder   20068 non-null  object 
 1   Date       20068 non-null  object 
 2   Time       20068 non-null  object 
 3   Path       20068 non-null  object 
 4   Municipio  20068 non-null  object 
 5   Vereda     20068 non-null  object 
 6   Latitud    20068 non-null  float64
 7   Longitud   20068 non-null  float64
 8   Elevacion  20068 non-null  int64  
 9   Habitat    20068 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.5+ MB


In [50]:
df_archivos

,Recorder,Date,Time,Path,Municipio,Vereda,Latitud,Longitud,Elevacion,Habitat
0,G03,2018/05/11,12:00:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
1,G03,2018/05/11,12:16:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
2,G03,2018/05/11,12:32:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
3,G03,2018/05/11,12:48:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
4,G03,2018/05/11,13:04:00,/content/drive/MyDrive/Jaguas_2018/G03_m/G03_2...,Alejandria,El Respaldo,6.41,-750.366,1322,Forest
...,...,...,...,...,...,...,...,...,...,...
20063,G58,2018/07/04,07:00:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.35,-749.696,1072,Forest
20064,G58,2018/07/04,06:45:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.35,-749.696,1072,Forest
20065,G58,2018/07/04,07:15:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.35,-749.696,1072,Forest
20066,G58,2018/07/04,07:30:00,/content/drive/MyDrive/Jaguas_2018/G58_m/G58_2...,San Rafael,Area proteccion embalse EPM,6.35,-749.696,1072,Forest


#**Exploracion Base De Datos**

In [ ]:
# Reproducir el archivo .wav
Audio(df_archivos["Path"][0], autoplay=True)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
len(df_archivos["Recorder"].value_counts().sort_index())

31